<a href="https://colab.research.google.com/github/HannaKi/priva_DL_HLT/blob/master/Text_Classification_task.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Text classification

This project is about text classification. You will develop a text classification system that identifies different kinds of online texts, such as news, blogs and opinionated texts. We will refer to these text categories as registers. If you want to learn more about online registers and their automatic identification, you can read, e.g., our paper [Toward Multilingual Identification of Online Registers] (https://www.aclweb.org/anthology/W19-6130/).

# Data and register labels
The data for this project consist of ~7500 documents with manual annotations on their register. You can download it from http://dl.turkunlp.org/TKO_8965-projects/classification/ . The documents are based on a (almost) random sample of the Finnish Internet. The registers are identified using a relatively detailed, hierarchical taxonomy. The taxonomy consists of 8 main categories that are divided into a large number of subregisters. The taxonomy is described at the end of this page. The table includes also the abbreviations that are used in the data.

The challenge with online documents is that it is not always easy to identify the specific registers categories of the documents. Furthermore, another issue is that a document may display characteristics of several registers. For instance, a blog post may simultaneously seem like a product review. To deal with these challenges, we have followed the following guidelines:
* For each document, the annotators have aimed at marking the specific subregister category. When this is possible, the document has two register labels: the subregister label and the main register label to which the subregister belongs. For instance, a document annotated as a news article would have the label NE for News and the corresponding higher level register label NA for Narrative. 
* In some cases, the document does not seem to fit any of the subregisters. In this case, the document can be given only one label: the main register label, such as NA for Narrative. 
* Some documents may display characteristics of several register categories. In this case, the annotator can mark several register labels for one single document. Consequently, the document may have up to four labels. This would be the case case if a document is annotated both as a Personal blog (subregister label PB + corresponding higher level register label NA) and Review (subregister label RV + corresponding higher level register label OP).


# Register classes and abbreviations

NA Narrative

* NE NA    New reports / news blogs
* SR NA    Sports reports
* PB NA    Personal blog
* HA NA    Historical article
* FC NA    Fiction
* TB NA    Travel blog
* CB NA    Community blogs
* OA NA    Online article

OP  Opinion
* OB OP  Personal opinion blogs
* RV OP  Reviews
* RS OP  Religious blogs/sermons
* AV OP  Advice

IN Informational description
* JD IN  Job description
* FA IN  FAQs
* DT IN  Description of a thing
* IB IN  Information blogs
* DP IN  Description of a person
* RA IN  Research articles
* LT IN  Legal terms / conditions
* CM IN  Course materials
* EN IN  Encyclopedia articles
* RP IN  Report

ID Interactive discussion
* DF ID  Discussion forums
* QA ID  Question-answer forums

HI  How-to/instructions
* RE HI  Recipes

IP IG  Informational persuasion
* DS IG  Description with intent to sell
* EB IG  News-opinion blogs / editorials

Lyrical LY
* PO LY  Poems
* SL LY  Songs

Spoken SP
* IT SP Interviews
* FS SP Formal speeches

Others OS
* MT OS Machine-translated / generated texts


# Preparations

Download and open data, explore it.

In [2]:
# Get rid of old tf at some point!

%tensorflow_version 1.x
# to run with old tf with which the code was made
# The default version of TensorFlow in Colab will switch to TensorFlow 2.x on the 27th of March, 2020.
# https://colab.research.google.com/notebooks/tensorflow_version.ipynb

import tensorflow
print(tensorflow.__version__)

TensorFlow is already loaded. Please restart the runtime to change versions.
2.2.0-rc2


In [3]:
# Download development data
!wget http://dl.turkunlp.org/TKO_8965-projects/classification/fincore-dev.tsv
# Download test data
!wget http://dl.turkunlp.org/TKO_8965-projects/classification/fincore-test.tsv
# Download train data
!wget http://dl.turkunlp.org/TKO_8965-projects/classification/fincore-train.tsv

--2020-04-13 10:37:27--  http://dl.turkunlp.org/TKO_8965-projects/classification/fincore-dev.tsv
Resolving dl.turkunlp.org (dl.turkunlp.org)... 195.148.30.23
Connecting to dl.turkunlp.org (dl.turkunlp.org)|195.148.30.23|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4035578 (3.8M) [application/octet-stream]
Saving to: ‘fincore-dev.tsv’

fincore-dev.tsv     100%[===================>]   3.85M  1.65MB/s    in 2.3s    

2020-04-13 10:37:30 (1.65 MB/s) - ‘fincore-dev.tsv’ saved [4035578/4035578]

--2020-04-13 10:37:32--  http://dl.turkunlp.org/TKO_8965-projects/classification/fincore-test.tsv
Resolving dl.turkunlp.org (dl.turkunlp.org)... 195.148.30.23
Connecting to dl.turkunlp.org (dl.turkunlp.org)|195.148.30.23|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 8512687 (8.1M) [application/octet-stream]
Saving to: ‘fincore-test.tsv’

fincore-test.tsv    100%[===================>]   8.12M  3.08MB/s    in 2.6s    

2020-04-13 10:37:36 (3.08 MB/

Data split
  - Train data - all training based on it (this includes the vectorizer!)
  - Development data - set the parameters (a.k.a validation data set)
  - Test data - used for nothing during training, produce final results



In [4]:
import pandas as pd

train = pd.read_csv('fincore-train.tsv', sep='\t', header=None)

train = train.sample(frac=1, random_state = 4) # suffle the data
train.columns = ['label','text']
print(train.head())
print(train.shape)

       label                                               text
3982  DS IG    Logistiikka Jenni Lindholm Laskutus Ritva Lie...
2640  RS OP    Tässä [ [ Ortodoksinen seminaari ( Joensuu ) ...
119   NE NA    Koulutuspaikka jokaiselle peruskoulun päättän...
4916  SR NA    1 Cardiff C–Everton Tasainen kohde . Cardiff ...
775   MT OS    Northrop Grumman Q4 2009 tulokset Northrop Gr...
(5295, 2)


In [5]:
dev = pd.read_csv('fincore-dev.tsv', sep='\t', header=None)
dev.columns = ['label','text']
print(dev.head())
print(dev.shape)

    label                                               text
0  OA NA    Luonnonhoito Maaperän siemenpankkia avattiin ...
1  DS IG    • Jokainen ripsi on erittäin kevyt ja muodolt...
2  DS IG    Mukavuudet Hotel Dila Vain muutaman metrin pä...
3  DF ID    Vastaa viestiin Otsikko Viesti ensin omaishoi...
4  OA NA    Dinosaur Jr 30.5.2010 Tavastia , Helsinki 198...
(756, 2)


In [6]:
test = pd.read_csv('fincore-test.tsv', sep='\t', header=None)
test.columns = ['label','text']
print(test.head())
print(test.shape)

test_text = test['text']
test_labels = test['label']

    label                                               text
0    HI     Tehkää nollaleimaus . Jos rekisteröinti onnis...
1    NA     1 kommenttia : Syyslomallelähtijät kirjoitti ...
2  DT IN    Ammattikoulutuksen perustana on ajatus siitä ...
3  DP IN    Ulkonäkö : Silveriä voisi kuvata tietyllä tap...
4  DT IN    Laulupelimannien puheenjohtajina ovat toimine...
(1513, 2)


In [8]:
# Gather features and labels of the data

# Separate text and the associated label
train_text = train['text']
train_labels = train['label']

print(train_text.head())
print(train_labels.head())
print()

dev_text = dev['text']
dev_labels = dev['label']

test_text = test['text']
test_labels = test['label']

labels = [train_labels, dev_labels, test_labels]
all_labels = pd.concat(labels)

print(all_labels.head(10))
print()
class_count = len(all_labels.unique())
print("Number of unique labels in data: ", class_count)

3982     Logistiikka Jenni Lindholm Laskutus Ritva Lie...
2640     Tässä [ [ Ortodoksinen seminaari ( Joensuu ) ...
119      Koulutuspaikka jokaiselle peruskoulun päättän...
4916     1 Cardiff C–Everton Tasainen kohde . Cardiff ...
775      Northrop Grumman Q4 2009 tulokset Northrop Gr...
Name: text, dtype: object
3982    DS IG 
2640    RS OP 
119     NE NA 
4916    SR NA 
775     MT OS 
Name: label, dtype: object

3982    DS IG 
2640    RS OP 
119     NE NA 
4916    SR NA 
775     MT OS 
5264      NA  
5040    DP IN 
3446    DS IG 
216     DF ID 
863     DT IN 
Name: label, dtype: object

Number of unique labels in data:  119


# Milestone 1.1: Bag-of-words classifier (multi-class)

Train a bag-of-words classifier to predict the register categories. In this milestone, the setting is multi-class, so the register label combinations form the classes, e.g. NA_NE and NA_NE_OP_OB. 

- Evaluate your model and report your results with different hyperparameters
- Ideas to try:
  - Different activation functions
  - Altering the learning rate
  - Use different optimizers
  - Adjusting the vocabulary size of the embeddings

- Activation functions and optimizers supported by Keras can be found here: https://keras.io/


Bow classifier is only interested in the multiplicity or appearance of words (or to be precise n-garms). Hence we loose the textual context and order of the words (n-grams). This inevitably leads to some information loss.

We will use CountVectorizer from sklearn package to transform out text data to numerical format with which our classifier is able to deal with. CountVectorizer converts the collection of text documents (our training data) to a matrix of token counts. Since we are only interested whether a particular word ot the vocabulary is in a single document or not, our vectorizer is set on "binary". 

In [9]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer(max_features = 97000, binary = True, ngram_range = (1,1))
# form feature matrix
train_feature_matrix = vectorizer.fit_transform(train_text)
dev_feature_matrix = vectorizer.fit_transform(dev_text)

print("shape of the training data: ", train_feature_matrix.shape)
print("shape of the development data: ", dev_feature_matrix.shape)

shape of the training data:  (5295, 97000)
shape of the development data:  (756, 97000)


The shape of the feature matrix tells us that we have 5295 items (documents) in our training data. The number of unique n-grams exceeds 97 000 but we are including only the first 97 000 most common of them. Since our CountVectorizer has parameter setting "ngram_range = 1, 1" this means we are forming the vector with unigrams, separate words or charachters.

## Label encoding

Next we will encode the labels. This means transforming the textula labels no numeric values, which our model is able to deal with. This step is made with LabelEncoder class.

In [10]:
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder() # Create the instance of LabelEncoder we use to turn class labels into integers

label_encoder.fit(all_labels) # encode labels to integers

train_numbers = label_encoder.transform(train_labels) 
dev_numbers = label_encoder.transform(dev_labels) 
test_numbers = label_encoder.transform(test_labels) 

print("Inverse transform gives unique labels in each data set: ", label_encoder.inverse_transform(train_numbers))
print("Sanity checks, do we have as many labels and texts in our data sets")
print(len(train_numbers), len(train_text))
print(len(dev_numbers), len(dev_text))
print(len(test_numbers), len(test_text))

Inverse transform gives unique labels in each data set:  ['DS IG ' 'RS OP ' 'NE NA ' ... 'DS IG ' 'MT OS ' 'MT OS ']
Sanity checks, do we have as many labels and texts in our data sets
5295 5295
756 756
1513 1513


In [0]:
# LabelEncoderin testailua ÄLÄ HÄVITÄ!!!!!!

# le = LabelEncoder()
# le.fit(["paris", "paris", "tokyo", "amsterdam"]) # encode tee koodaus
# print("Koodaukset: ", list(le.classes_))
# print()
# le.transform(["tokyo", "tokyo", "paris"]) # käytä koodausta arvojen transformointiin --> numeeriset arvot
# num = le.transform(["tokyo", "tokyo", "paris"])
# print("Sovitetulla encoderilla tuotetut numeeriset arvot muuttujalistasta: ", num)
# print()
# print("Inverse transform: ", list(le.inverse_transform([2, 2, 1])))
# print()
# test = le.fit_transform(["paris", "paris", "tokyo", "amsterdam"]) # encode and transform
# print("Koodit kolmella", test)

# le.fit(["paris", "paris", "tokyo", "amsterdam", "helsinki"])
# #print(list(le.classes_))

# test = le.fit_transform(["paris", "helsinki", "tokyo", "amsterdam"]) # encode and transform
# print("Koodit neljällä", test)

# print(list(le.inverse_transform([2, 2, 1])))


Now the data is prepared and we move on to building the classifier itself.

In [18]:
import keras
from keras.models import Model
from keras.layers import Input, Dense

# from tensorflow.python.keras.models import Model
# from tensorflow.python.keras.layers import Input, Dense

example_count, feature_count = train_feature_matrix.shape

inp = Input(shape = (feature_count, ))                  # Tuple. The size of the inputlayer is the number of the vectors
hidden = Dense(200, activation="tanh")(inp)             # Non-linear activation function. tanh or relu? 
outp = Dense(class_count, activation="softmax")(hidden) # As many output possibilities as we have input classes. ALL THE POSSIBLE CLASSES!!?!??! 
                                                        # Softmax: produces probability distribution of the classes
bow_model = Model(inputs=[inp], outputs=[outp])

bow_model.summary()

Model: "model_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         (None, 97000)             0         
_________________________________________________________________
dense_5 (Dense)              (None, 200)               19400200  
_________________________________________________________________
dense_6 (Dense)              (None, 119)               23919     
Total params: 19,424,119
Trainable params: 19,424,119
Non-trainable params: 0
_________________________________________________________________


In [0]:
bow_model.compile(optimizer="adam", loss = "sparse_categorical_crossentropy", metrics = ['accuracy'])

Now we will fit the data. Here we will also need the validation data.

batch_size kuinka monta inputtia kerralla sisaan. jokaisen batchin jalkeen paivitetaan painokertoimet gradientien keskiarvolla
epochs kuinka monta kertaa mennaan lapi koko data
validation_split: kuinka paljon dataa kaytetaan accuracyn laskemiseen

- jos näyttää sille, että mallin oppiminen paranisi vaikka malli on jo treenattu (val_acc kehittyy paremmaksi), lisaa epocheja. Käytä early stoppingia estään ylisovittaminen

## Fitting BOW-classifier

Let's try with different optimizers available in Keras.

In [16]:
from keras.callbacks import ModelCheckpoint, EarlyStopping

stop_cb = EarlyStopping(monitor = 'val_accuracy', patience=3, verbose=1, mode='auto', baseline=None, restore_best_weights=True)

# https://www.tensorflow.org/api_docs/python/tf/keras/optimizers/Ftrl
# Lisää tähän loopiin eri learningratet?
ops = ('Adadelta', 'Adagrad', 'Adam', 'SGD', 'RMSProp', 'Adamax', 'Nadam')

# tee uusi muuttuja, jossa LR: liitetty loopin avulla optimizeriin!

for op in ops:
  print(op)
  print()
  bow_model.compile(optimizer = op, loss="sparse_categorical_crossentropy", metrics=['accuracy'])
  bow_history = bow_model.fit(train_feature_matrix, train_numbers, batch_size=100, 
                 verbose=1, epochs=25, validation_data=(dev_feature_matrix, dev_numbers), callbacks=[stop_cb])
  print()

Adadelta

Train on 5295 samples, validate on 756 samples
Epoch 1/25
5295/5295 [==============================] - 5s 980us/step - loss: 0.6906 - accuracy: 0.8844 - val_loss: 3.8670 - val_accuracy: 0.0873
Epoch 2/25
5295/5295 [==============================] - 5s 860us/step - loss: 0.3708 - accuracy: 0.9477 - val_loss: 3.8192 - val_accuracy: 0.0939
Epoch 3/25
5295/5295 [==============================] - 4s 845us/step - loss: 0.2163 - accuracy: 0.9741 - val_loss: 3.8148 - val_accuracy: 0.0992
Epoch 4/25
5295/5295 [==============================] - 4s 833us/step - loss: 0.1357 - accuracy: 0.9873 - val_loss: 3.8126 - val_accuracy: 0.0966
Epoch 5/25
5295/5295 [==============================] - 4s 817us/step - loss: 0.0885 - accuracy: 0.9928 - val_loss: 3.8117 - val_accuracy: 0.1005
Epoch 6/25
5295/5295 [==============================] - 4s 809us/step - loss: 0.0609 - accuracy: 0.9955 - val_loss: 3.8164 - val_accuracy: 0.0979
Epoch 7/25
5295/5295 [==============================] - 4s 809us/st

In [33]:
# Choose best OP and LR and fit the model
bow_model.compile(optimizer = 'adam', loss="sparse_categorical_crossentropy", metrics=['accuracy'])

bow_history = bow_model.fit(train_feature_matrix, train_numbers, batch_size=100, 
                 verbose=1, epochs=25, validation_data=(dev_feature_matrix, dev_numbers), callbacks=[stop_cb])

Train on 5295 samples, validate on 756 samples
Epoch 1/25
5295/5295 [==============================] - 4s 801us/step - loss: 0.2646 - accuracy: 0.9632 - val_loss: 4.0338 - val_accuracy: 0.0899
Epoch 2/25
5295/5295 [==============================] - 4s 787us/step - loss: 0.0516 - accuracy: 0.9949 - val_loss: 4.1494 - val_accuracy: 0.0939
Epoch 3/25
5295/5295 [==============================] - 4s 779us/step - loss: 0.0221 - accuracy: 0.9964 - val_loss: 4.2196 - val_accuracy: 0.0939
Epoch 4/25
5295/5295 [==============================] - 4s 774us/step - loss: 0.0168 - accuracy: 0.9966 - val_loss: 4.2510 - val_accuracy: 0.0939
Epoch 5/25
5295/5295 [==============================] - 4s 769us/step - loss: 0.0113 - accuracy: 0.9974 - val_loss: 4.2842 - val_accuracy: 0.0966
Epoch 6/25
5295/5295 [==============================] - 4s 774us/step - loss: 0.0109 - accuracy: 0.9970 - val_loss: 4.3118 - val_accuracy: 0.0952
Epoch 7/25
5295/5295 [==============================] - 4s 777us/step - loss:

In [0]:
# import numpy as np

# predictions = model.predict(dev_feature_matrix)
# pred_classes = np.argmax(predictions,axis=-1)
# for pred, correct, txt_line in zip(pred_classes, dev_labels, dev_text):
#     pred_label=label_encoder.classes_[pred]
#     if pred_label!=correct:
#         print("Prediction:",pred_label,"Correct:",correct,"Text:",txt_line)

In [21]:
# form feature matrix for test data set
test_feature_matrix = vectorizer.fit_transform(test_text)

print(test_feature_matrix.shape)

(1513, 97000)


In [0]:
# # Code for plot from:
# # http://scikit-learn.org/stable/auto_examples/model_selection/plot_confusion_matrix.html#sphx-glr-auto-examples-model-selection-plot-confusion-matrix-py

# import matplotlib.pyplot as plt
# import itertools
# %matplotlib inline 

# def plot_confusion_matrix(cm, classes,
#                           normalize = False,
#                           title = 'Confusion matrix',
#                           cmap = plt.cm.Blues):
    
    
#     """
#     This function prints and plots the confusion matrix.
#     Normalization can be applied by setting `normalize=True`.
#     """
#     if normalize:
#         cm = cm.astype('float') / cm.sum(axis = 1)[:, np.newaxis]
#         print("Normalized confusion matrix")
#     else:
#         print('Confusion matrix, without normalization')

#     print(cm)

#     plt.imshow(cm, interpolation='nearest', cmap = cmap)
#     plt.title(title)
#     plt.colorbar()
#     tick_marks = np.arange(len(classes))
#     plt.xticks(tick_marks, classes, rotation=45)
#     plt.yticks(tick_marks, classes)

#     fmt = '.2f' if normalize else 'd'
#     thresh = cm.max() / 2.
#     for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
#         plt.text(j, i, format(cm[i, j], fmt),
#                  horizontalalignment = "center",
#                  color="white" if cm[i, j] > thresh else "black")

#     plt.ylabel('True label')
#     plt.xlabel('Predicted label')
#     plt.tight_layout() 

In [23]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

#print("Network output=",model.predict(test_feature_matrix))
predictions = np.argmax(bow_model.predict(test_feature_matrix), axis=1) # np.argmax gives the index which has the highest value e.g. class
print("true labels: \n", test_labels)
# target_labels = label_encoder.inverse_transform(list(target))
predicted_labels = label_encoder.inverse_transform(list(predictions))
print("predicted labels: \n", predicted_labels)
print()
print("Classification accuracy: ", round(accuracy_score(test_labels, predicted_labels)*100,1), "percent")

# print(classification_report(test_labels, predicted_labels))

true labels: 
 0         HI  
1         NA  
2       DT IN 
3       DP IN 
4       DT IN 
         ...  
1508    OB OP 
1509    MT OS 
1510    OA NA 
1511    PB NA 
1512    MT OS 
Name: label, Length: 1513, dtype: object
predicted labels: 
 ['NE NA IP IG ' 'DP IN ' 'CB NA EB IG ' ... 'ID  FC NA ' 'DS IG ' 'PO LY ']

Classification accuracy:  0.7 percent


Why does the model perform so badly? 119 classes. 

In [30]:
import numpy as np
print("Number of unique labels in")
print("-train data: ", len(np.unique(train_numbers)))
print("-development data: ", len(np.unique(dev_numbers)))
print("-test data: ", len(np.unique(test_numbers)))
print()
inter = np.intersect1d(train_numbers, dev_numbers)
inter2 = np.intersect1d(train_numbers, test_numbers)
print("Number of shared labels in")
print("-train and development data: ", len(inter))
print("-train and test data: ", len(inter2))

Number of unique labels in
-train data:  100
-development data:  58
-test data:  70

Number of shared labels in
-train and development data:  52
-train and test data:  57


In [0]:
# cnf_matrix = confusion_matrix(test_labels, predicted_labels)

# # Confusion matrix has the true labels on rows, and predicted labels on columns in sorted order
# print(cnf_matrix)

In [0]:
# # Plot confusion matrix
# plt.figure()
# plot_confusion_matrix(cnf_matrix, classes = all_labels, normalize = False)

# plt.show()

In [0]:
# # np-argmaxin testailua ÄLÄ HÄVITÄ!!!!!!
# print(predictions[0])
# print(model.predict(test_feature_matrix)[0][25])
# print(model.predict(test_feature_matrix)[0])
# print(sum(model.predict(test_feature_matrix)[0]))

# Milestone 1.2: Recurrent Neural Network Classifier (multi-class)

Modify your codes from milestone 1.1 to use recurrent neural networks (e.g. LSTM or biLSTM) in the classifier. Evaluate your model and report your results with different hyperparameters.

For RNN-calssifier we use Tokenizer which turns tokens, in our case the words of training data to integers. 

## Tokenizing

In [0]:
from keras.preprocessing.text import Tokenizer

tokenizer = Tokenizer(
    num_words=97000, # max num of most common words
)

tokenizer.fit_on_texts(train_text)

In [21]:
from pprint import pprint    # pretty-printer

def truncate_dict(d, count=10):
    # Returns at most count items from the given dictionary.  
    return dict(i for i, _ in zip(d.items(), range(count)))

# Check if 0 is in the index, and print examples of the mapping
# 0 is reserved for padding!
print(tokenizer.word_index.get(0))
pprint(truncate_dict(tokenizer.word_index))

None
{'ei': 3,
 'että': 4,
 'ja': 1,
 'kun': 9,
 'mutta': 8,
 'oli': 7,
 'on': 2,
 'ovat': 10,
 'se': 6,
 'tai': 5}


In [22]:
train_sequences = tokenizer.texts_to_sequences(train_text)

print(len(train_sequences)) 

# Print an example text, its corresponding sequence, and the tokens it represents
print('Text:', train_text.head(1)[0:200]) # first item of the suffled data (index not 0!)
print('Sequence:', train_sequences[0][:10])
print('Mapped back:', [tokenizer.index_word[i] for i in train_sequences[0][:10]])

5295
Text: 3982     Logistiikka Jenni Lindholm Laskutus Ritva Lie...
Name: text, dtype: object
Sequence: [21429, 13274, 40282, 11697, 15340, 796, 657, 388, 12826, 3002]
Mapped back: ['logistiikka', 'jenni', 'lindholm', 'laskutus', 'ritva', 'ota', 'yhteyttä', 'nimi', 'puhelinnumero', 'sähköposti']


In [23]:
lengths = [len(s) for s in train_sequences]
print('Lengths:', lengths[:10], 'min:', min(lengths), 'max:', max(lengths), 'mean:', np.mean(lengths))

Lengths: [384, 921, 158, 194, 932, 388, 123, 384, 1577, 167] min: 0 max: 79136 mean: 583.0457034938621


## Padding

Since Keras demands for all of the input items (separate documents of our training data) to have the same length, we need to "pad" all but the longest document by filling in the "missing" number of words with zeros.


In [24]:
from keras.preprocessing.sequence import pad_sequences

sequence_length = 585 # based on mean value of input length: we will cut sequences longer than this and pad with zeros sequeces shorter than this

padded_X = pad_sequences(
    train_sequences,
    maxlen = sequence_length, 
    value=0
)

print(padded_X.shape)

(5295, 585)


In [25]:
# Prepare model development data

dev_sequences = tokenizer.texts_to_sequences(dev_text)
padded_dev = pad_sequences(
    dev_sequences,
    maxlen = sequence_length, 
    value=0
)

print(padded_dev.shape)

(756, 585)


# Build LSTM RNN

# HUOMAA!!!
Tämä teksti alkuperäisestä RNN-classification notebookista!

We define a basic RNN model that takes the RNN cell class (RNN_class) as an argument:

- input: sequence of sequence_length integers corresponding to words
- embedding: randomly initialized mapping from integers to embedding_dim-dimensional vectors
- rnn: recurrent neural network with rnn_units-dimensional state
- output: num_classes-dimensional fully connected layer with softmax activation

# KATSO NÄITÄ!
We're intentionally leaving out a few fairly obvious things that would be expected to help here, including

- Any form of regularization, e.g. dropout
- Initializing the embeddings with pre-trained word vectors (ks. fasttext)
- Masking to ignore padding (see Masking and padding with Keras)


In [0]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, LSTM, Dense

# We'll use these model parameters for all of our examples here.
embedding_dim = 50 # input vector
rnn_units = 100

def build_rnn_model(RNN_class, sequence_length, vocab_size, num_classes):
    input_ = Input(shape=(sequence_length,))
    embedding = Embedding(vocab_size, embedding_dim)(input_) # randomly initialized. Layer turns positive integers (indexes) into dense vectors of fixed size
    rnn = RNN_class(rnn_units)(embedding) # can support different RNNs
    output = Dense(num_classes, activation='softmax')(rnn)
    return Model(inputs=[input_], outputs=[output])

sequence_length = padded_X.shape[1]
vocab_size = tokenizer.num_words
num_classes = len(label_encoder.classes_)

In [60]:
len(label_encoder.classes_)

119

In [52]:
lstm_model = build_rnn_model(LSTM, sequence_length, vocab_size, num_classes)

lstm_model.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         [(None, 585)]             0         
_________________________________________________________________
embedding_5 (Embedding)      (None, 585, 50)           4850000   
_________________________________________________________________
lstm_4 (LSTM)                (None, 100)               60400     
_________________________________________________________________
dense_3 (Dense)              (None, 119)               12019     
Total params: 4,922,419
Trainable params: 4,922,419
Non-trainable params: 0
_________________________________________________________________


In [0]:
lstm_model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

epochs = 25
batch_size = 100

In [31]:
lstm_history = lstm_model.fit(padded_X, train_numbers, epochs = epochs, batch_size = batch_size, validation_data=(padded_dev, dev_numbers), callbacks=[stop_cb])


Train on 5295 samples, validate on 756 samples
Epoch 1/25
5295/5295 [==============================] - 45s 9ms/sample - loss: 2.5684 - acc: 0.3707 - val_loss: 2.7282 - val_acc: 0.3135
Epoch 2/25
5295/5295 [==============================] - 44s 8ms/sample - loss: 2.2642 - acc: 0.4274 - val_loss: 2.6562 - val_acc: 0.3479
Epoch 3/25
5295/5295 [==============================] - 45s 8ms/sample - loss: 1.9368 - acc: 0.5025 - val_loss: 2.6190 - val_acc: 0.3426
Epoch 4/25
5295/5295 [==============================] - 44s 8ms/sample - loss: 1.6130 - acc: 0.5847 - val_loss: 2.7263 - val_acc: 0.3585
Epoch 5/25
5295/5295 [==============================] - 44s 8ms/sample - loss: 1.3635 - acc: 0.6591 - val_loss: 2.8424 - val_acc: 0.3519
Epoch 6/25
5295/5295 [==============================] - 45s 8ms/sample - loss: 1.1233 - acc: 0.7212 - val_loss: 2.9840 - val_acc: 0.3161
Epoch 7/25
5295/5295 [==============================] - 45s 8ms/sample - loss: 0.9020 - acc: 0.7847 - val_loss: 3.2058 - val_acc: 0

# Bidirectional LSTM RNN

In [0]:
# Tämä ei toimi! Virhe nimeämisestä, selvittämättä!
# from keras.layers import Bidirectional as Bi

# def build_bi_rnn_model(RNN_class, sequence_length, vocab_size, num_classes):
#     input_ = Input(shape=(sequence_length,))
#     embedding = Embedding(vocab_size, embedding_dim)(input_) # randomly initialized
#     rnn = Bi(RNN_class(rnn_units))(embedding) # can support different RNNs
#     output = Dense(num_classes, activation='softmax')(rnn)
#     return Model(inputs=[input_], outputs=[output])


In [0]:
# lstm_bi_model = build_bi_rnn_model(LSTM, sequence_length, vocab_size, num_classes)

# lstm_bi_model.summary()

In [0]:
# lstm_bi_model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [0]:
# from keras.models import Sequential
# from keras.layers import Dense, Embedding, Activation, Bidirectional, LSTM

# # model = Sequential()
# # model.add(Bidirectional(LSTM(10, return_sequences=True),
# #                         input_shape=(5, 10)))
# # model.add(Bidirectional(LSTM(10)))
# # model.add(Dense(5))
# # model.add(Activation('softmax'))
# # model.compile(loss='categorical_crossentropy', optimizer='rmsprop')

# model = Sequential()
# model.add(Dense(90, input_dim=sequence_length))
# model.add(Embedding(vocab_size, embedding_dim))
# model.add(Bidirectional(LSTM(rnn_units)))
# model.add(Dense(num_classes))
# model.add(Activation('softmax'))

# model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# model.summary()

In [57]:
from keras.models import Sequential
from keras.layers import Dense, Embedding, Activation, Bidirectional, LSTM

# def build_bi_rnn_model(RNN_class, sequence_length, vocab_size, num_classes):
#     input_ = Input(shape=(sequence_length,))
#     embedding = Embedding(vocab_size, embedding_dim)(input_) # randomly initialized
#     rnn = Bi(RNN_class(rnn_units))(embedding) # can support different RNNs
#     output = Dense(num_classes, activation='softmax')(rnn)
#     return Model(inputs=[input_], outputs=[output])

model = Sequential()
# model.add(Input(shape=(sequence_length,)
model.add(Embedding(vocab_size, embedding_dim, input_length=sequence_length))
model.add(Bidirectional(LSTM(rnn_units))) # bidirectional: num of neurons gets doubled
model.add(Dense(num_classes))
model.add(Activation('softmax'))

model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

model.summary()

Model: "sequential_16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_9 (Embedding)      (None, 585, 50)           4850000   
_________________________________________________________________
bidirectional_7 (Bidirection (None, 200)               120800    
_________________________________________________________________
dense_19 (Dense)             (None, 119)               23919     
_________________________________________________________________
activation_6 (Activation)    (None, 119)               0         
Total params: 4,994,719
Trainable params: 4,994,719
Non-trainable params: 0
_________________________________________________________________


In [63]:
from keras.models import Sequential
from keras.layers import Dense, Embedding, Activation, Bidirectional, LSTM

# def build_bi_rnn_model(RNN_class, sequence_length, vocab_size, num_classes):
#     input_ = Input(shape=(sequence_length,))
#     embedding = Embedding(vocab_size, embedding_dim)(input_) # randomly initialized
#     rnn = Bi(RNN_class(rnn_units))(embedding) # can support different RNNs
#     output = Dense(num_classes, activation='softmax')(rnn)
#     return Model(inputs=[input_], outputs=[output])

model = Sequential()
model.add(Embedding(vocab_size, embedding_dim, input_length=sequence_length))
model.add(Bidirectional(LSTM(rnn_units))) # bidirectional: num of neurons gets doubled
model.add(Dense(num_classes, activation = 'softmax'))

model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

model.summary()

Model: "sequential_19"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_12 (Embedding)     (None, 585, 50)           4850000   
_________________________________________________________________
bidirectional_10 (Bidirectio (None, 200)               120800    
_________________________________________________________________
dense_22 (Dense)             (None, 119)               23919     
Total params: 4,994,719
Trainable params: 4,994,719
Non-trainable params: 0
_________________________________________________________________


In [64]:
hist_bi_lstm = model.fit(padded_X, train_numbers, epochs = epochs, batch_size = batch_size, validation_data=(padded_dev, dev_numbers), callbacks=[stop_cb])

Train on 5295 samples, validate on 756 samples
Epoch 1/25
5295/5295 [==============================] - 54s 10ms/step - loss: 3.5674 - accuracy: 0.1063 - val_loss: 3.1816 - val_accuracy: 0.1270
Epoch 2/25
5295/5295 [==============================] - 52s 10ms/step - loss: 3.1508 - accuracy: 0.1569 - val_loss: 3.1199 - val_accuracy: 0.1852
Epoch 3/25
5295/5295 [==============================] - 52s 10ms/step - loss: 3.0942 - accuracy: 0.2000 - val_loss: 3.0066 - val_accuracy: 0.2077
Epoch 4/25
5295/5295 [==============================] - 53s 10ms/step - loss: 2.7013 - accuracy: 0.3290 - val_loss: 2.6001 - val_accuracy: 0.3651
Epoch 5/25
5295/5295 [==============================] - 53s 10ms/step - loss: 2.3044 - accuracy: 0.4344 - val_loss: 2.5857 - val_accuracy: 0.3492
Epoch 6/25
5295/5295 [==============================] - 52s 10ms/step - loss: 1.8914 - accuracy: 0.5265 - val_loss: 2.5611 - val_accuracy: 0.3810
Epoch 7/25
5295/5295 [==============================] - 53s 10ms/step - loss:

# Milestone 2.1: Deep contextual representations with Bert (multi-class)
Train a Bert classifier to predict the register categories. Similar to Milestone 1, the setting is multi-class, and the evaluations should include results with different hyperparameters.

# Milestone 2.2: Error analysis
Compare the errors made by the classifiers you have trained from milestones 1 and 2.1. Are there any patterns? How do the errors one model makes differ from those made by another.

# Milestone 3.1: Bert (multi-LABEL)
Train two multi-label classifiers, one using non-deep contextual representations, the other using Bert. In this setting, each label is assigned independently. Do hyperparameter optimization on these classifiers.

# Milestone 3.2: Model comparison
Compare the results of these two classifiers. Do the two models predict in the same way? Analyze the predictions in terms of label-specific differences.

# Register classes and abbreviations
